In [1]:
from kubernetes import client, config, watch
import uuid

# config.kube_config.load_kube_config("minikube_config.yml")
config.kube_config.load_kube_config("config.yml")
core_v1 = client.CoreV1Api()  #用于操作核心v1 API组中的资源，如Pod、Service、Node、Namespace等
batch_v1 = client.BatchV1Api()  #用于操作Job等资源
apps_v1 = client.AppsV1Api()  #用于操作Deployment、DaemonSet、ReplicaSet和StatefulSet
custom_api = client.CustomObjectsApi()  #用于操作CRD

In [2]:
nginx_pod_name = 'nginx-{}'.format(uuid.uuid4())
nginx_pod = {
    'apiVersion': 'v1',
    'kind': 'Pod',
    'metadata': {
        'name': nginx_pod_name,
        'labels': {
            "app": "nginx"
        }
    },
    'spec': {
        'containers': [
            {
                'name': 'my-container',
                'image': 'nginx:1.7.9',
                "ports": [
                    {
                        "containerPort": 80
                    }
                ]
            }
        ],
        #指定调度节点
        # 'nodeSelector': {
        #     'kubernetes.io/hostname': 'bigdata4'
        # }
    }
}
#创建
core_v1.create_namespaced_pod(body=nginx_pod, namespace="default")
#监控
close = True
try:
    w = watch.Watch()
    # 监控default命名空间中的pod
    for event in w.stream(core_v1.list_namespaced_pod, namespace="default"):
        print("event: %s %s" % (event['type'], event['object'].metadata.name))
        if event['object'].metadata.name == nginx_pod_name and event['type'] == "DELETED":
            print("pod:{} life end,exit watch".format(nginx_pod_name))
            w.stop()
            close = False
            break
except Exception as e:
    e.with_traceback()
finally:
    if close:
        core_v1.delete_namespaced_pod(name=nginx_pod_name, namespace="default")

event: ADDED nginx-9401e806-12af-4026-85af-6e52facd51e3
event: MODIFIED nginx-9401e806-12af-4026-85af-6e52facd51e3
event: MODIFIED nginx-9401e806-12af-4026-85af-6e52facd51e3
event: MODIFIED nginx-9401e806-12af-4026-85af-6e52facd51e3
event: MODIFIED nginx-9401e806-12af-4026-85af-6e52facd51e3
event: MODIFIED nginx-9401e806-12af-4026-85af-6e52facd51e3
event: MODIFIED nginx-9401e806-12af-4026-85af-6e52facd51e3
event: DELETED nginx-9401e806-12af-4026-85af-6e52facd51e3
pod:nginx-9401e806-12af-4026-85af-6e52facd51e3 life end,exit watch


In [7]:
group = "stable.example.com"
version = "v1"
plural = "crontabs"
namespace = "default"
try:
    w = watch.Watch()
    for event in w.stream(custom_api.list_namespaced_custom_object, group=group, version=version, namespace=namespace,
                          plural=plural):
        print("event: %s %s" % (event['type'], event["object"]["metadata"]["name"]))
        if event['type'] == "DELETED":
            w.stop()
            break
except Exception as e:
    e.with_traceback()

event: ADDED my-new-cron-object
event: DELETED my-new-cron-object
